In [ ]:
import numpy as np
import scipy.io
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, r2_score

from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector

from sklearn.preprocessing import StandardScaler


# 1) Load QM7, Flatten 23x23 Coulomb Matrices => 529 features


def load_qm7_coulomb_flat(matfile='qm7.mat'):
    """
    Loads the QM7 dataset from `matfile`.
    For each 23x23 Coulomb matrix, flattens into 529 features.
    Returns:
      X_flat: (N, 529) array of flattened Coulomb matrices
      Y:      (N,) array of target energies
    """
    data = scipy.io.loadmat(matfile)
    C_matrices = data['X']  # shape: (N, 23, 23)
    energies   = data['T'].ravel()  # shape: (N,)

    N = C_matrices.shape[0]
    X_list = []
    for i in range(N):
        # Flatten 23x23 => 529 features
        M_flat = C_matrices[i].flatten()
        X_list.append(M_flat)
    X_flat = np.array(X_list)  # shape (N, 529)
    return X_flat, energies


# 2) Amplitude Encoding Helper


def amplitude_encoding_circuit(vec, num_qubits):
    """
    Creates a circuit that amplitude-encodes 'vec' into 'num_qubits'.
      - Pads to length 2^num_qubits
      - Normalizes
      - Use qc.initialize(...)
    Returns a QuantumCircuit.
    """
    dim = 2**num_qubits
    qc = QuantumCircuit(num_qubits)

    padded = np.zeros(dim, dtype=complex)
    length = len(vec)
    if length > dim:
        raise ValueError(f"Cannot encode {length} features in {num_qubits} qubits (max {dim}).")

    padded[:length] = vec
    norm = np.linalg.norm(padded)
    if norm < 1e-9:
        padded[0] = 1.0
        norm = 1.0
    padded /= norm

    qc.initialize(padded, range(num_qubits))
    return qc

def build_statevectors(X, num_qubits):
    """
    For each row in X (shape (N, F)), it builds amplitude encoding,
    then retrieves the final statevector as a (2^num_qubits,) array.
    Returns sv_data of shape (N, 2^num_qubits) complex.
    """
    N = X.shape[0]
    dim = 2**num_qubits
    sv_data = np.zeros((N, dim), dtype=complex)
    for i in range(N):
        qc = amplitude_encoding_circuit(X[i], num_qubits)
        sv = Statevector.from_instruction(qc)
        sv_data[i] = sv.data
    return sv_data

def compute_kernel_statevector(sv_data):
    """
    Given shape (N, 2^num_qubits),
    computes NxN kernel K where K[i,j] = |< sv_i, sv_j >|^2.
    """
    N = sv_data.shape[0]
    K = np.zeros((N, N))
    for i in range(N):
        for j in range(i, N):
            ov = np.vdot(sv_data[i], sv_data[j])  # conj(sv[i]) dot sv[j]
            ov_sq = abs(ov)**2
            K[i,j] = ov_sq
            K[j,i] = ov_sq
    return K


# 3) Main: Feature + Target Scaling + Amplitude Encoding


def main_qsvr_scaled_coulomb_matrix_demo(
    matfile='qm7.mat',
    subset_size=600,
    test_size=0.2,
    num_qubits=10,
    random_seed=42
):
    """
    1) Loads dataset, flatten => 529 features
    2) Subsamples
    3) Scales features with StandardScaler
    4) Scales targets (subtract mean, divide std)
    5) Train/test split
    6) Amplitude encode -> direct statevector kernel
    7) Fits SVR
    8) inverts target transform, measures final MAE, R^2
    """
    # Load flattened data
    X_all, Y_all = load_qm7_coulomb_flat(matfile)
    Ntotal = len(X_all)
    print(f"Loaded {Ntotal} molecules with flattened 23x23 => 529 features.")

    # Subsample
    np.random.seed(random_seed)
    idxs = np.random.choice(Ntotal, size=subset_size, replace=False)
    X_sub = X_all[idxs]
    Y_sub = Y_all[idxs]

    # (1) Scale features
    scaler = StandardScaler()
    X_sub_scaled = scaler.fit_transform(X_sub)  # shape (subset_size, 529)

    # (2) Scale targets
    Y_mean = np.mean(Y_sub)
    Y_std  = np.std(Y_sub)
    Y_sub_scaled = (Y_sub - Y_mean)/Y_std

    # Train/test
    X_train, X_test, Y_train_scaled, Y_test_scaled = train_test_split(
        X_sub_scaled,
        Y_sub_scaled,
        test_size=test_size,
        random_state=random_seed
    )
    print(f"Train size={X_train.shape[0]}, Test size={X_test.shape[0]}")
    print(f"Amplitude encoding with {num_qubits} qubits => dimension=2^{num_qubits} >= 529.\n")

    # Build statevectors for train
    print("Building amplitude-encoded statevectors for training set...")
    sv_train = build_statevectors(X_train, num_qubits=num_qubits)
    print("Computing training kernel (direct overlap) ...")
    K_train = compute_kernel_statevector(sv_train)

    # Fit SVR with precomputed kernel
    svr = SVR(kernel='precomputed', C=10.0, epsilon=1.0)
    svr.fit(K_train, Y_train_scaled)

    # Build statevectors for test
    print("\nBuilding amplitude-encoded statevectors for test set...")
    sv_test = build_statevectors(X_test, num_qubits=num_qubits)

    # Cross-kernel test x train
    N_train = X_train.shape[0]
    N_test  = X_test.shape[0]
    K_test = np.zeros((N_test, N_train))
    for i in range(N_test):
        for j in range(N_train):
            ov = np.vdot(sv_test[i], sv_train[j])
            K_test[i,j] = abs(ov)**2

    # Predict (in scaled space)
    Y_pred_scaled = svr.predict(K_test)

    # Invert target transform
    Y_pred = Y_pred_scaled * Y_std + Y_mean
    Y_test = Y_test_scaled * Y_std + Y_mean

    # Evaluate
    mae = mean_absolute_error(Y_test, Y_pred)
    r2  = r2_score(Y_test, Y_pred)

    print("\nQuantum SVR (Amp Encoding, Full Coulomb, Scaled) Results:")
    print(f"  Subset size: {subset_size}")
    print(f"  MAE = {mae:.3f}")
    print(f"  R^2 = {r2:.3f}")


if __name__ == "__main__":
    # Example usage
    main_qsvr_scaled_coulomb_matrix_demo(
        matfile='qm7.mat',
        subset_size=1000,
        test_size=0.2,
        num_qubits=10,
        random_seed=42
    )
